In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load dataset
df = pd.read_csv('CP_count_Ha.csv')


In [2]:

df

,InChIKey,737823,737824,688422,688810,688812,688816,688724,688739,688759,...,Cells_Neighbors_FirstClosestObjectNumber_Adjacent,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number
0,IENZQIKPVFGBNW-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
1,GSDSWSVVBLHKDQ-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,3.030769,3.807692,3.807692,3.607692,3.607692,3.607692,3.607692,3.123077,4.588462,3.607692
2,CGIGDMFJXJATDK-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-6.011719,-5.312500,-5.312500,-6.164062,-6.164062,-6.164062,-6.164062,-5.976562,-6.132812,-6.164062
3,DSXXEELGXBCYNQ-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-31.820312,-31.812500,-31.812500,-31.828125,-31.828125,-31.828125,-31.828125,-31.835938,-32.304688,-31.828125
4,MYSWGUAQZAJSOK-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,4.070312,3.593750,3.593750,3.617188,3.617188,3.617188,3.617188,4.078125,3.164062,3.617188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10521,YAJYINBQFXCAPI-WENCSYSZSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-3.171875,-3.062500,-3.062500,-3.625000,-3.632812,-3.625000,-3.625000,-3.125000,-2.953125,-3.632812
10522,DIBJIDYNEHVBAV-KFRKLSNLSA-N,-1,-1,-1,-1,-1,-1,-1,0,-1,...,-2.031250,-1.648438,-1.648438,-3.046875,-3.046875,-3.046875,-3.046875,-2.468750,-2.132812,-3.046875
10523,YAJYINBQFXCAPI-VPYPWEPUSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-0.804688,-2.265625,-2.265625,-1.875000,-1.875000,-1.875000,-1.875000,-1.351562,-2.718750,-1.875000
10524,YAJYINBQFXCAPI-RZIGYZOXSA-N,-1,-1,-1,0,-1,-1,-1,-1,0,...,0.820312,0.843750,0.843750,1.796875,1.796875,1.796875,1.796875,1.101562,1.281250,1.796875


In [3]:
import pandas as pd
from tqdm import tqdm

# Initialize a list to collect the results
melts = []

# Iterate over the rows of the DataFrame
for row in tqdm(df.itertuples(index=False)):
    inchikey = row.InChIKey
    for i, value in enumerate(row[1:], start=1):
        if value in [0, 1]:
            melts.append({"InChIKey": inchikey, "Task": df.columns[i], "Label": value})

# Convert the collected results to a DataFrame
melt = pd.DataFrame(melts)

# Display the result
melt


10526it [00:01, 8616.17it/s]


,InChIKey,Task,Label
0,IENZQIKPVFGBNW-UHFFFAOYSA-N,737822,0.0
1,IENZQIKPVFGBNW-UHFFFAOYSA-N,737826,0.0
2,IENZQIKPVFGBNW-UHFFFAOYSA-N,737823_1,0.0
3,IENZQIKPVFGBNW-UHFFFAOYSA-N,737824_1,0.0
4,IENZQIKPVFGBNW-UHFFFAOYSA-N,1495242,0.0
...,...,...,...
54777,YAJYINBQFXCAPI-RZIGYZOXSA-N,688653,0.0
54778,YAJYINBQFXCAPI-RZIGYZOXSA-N,688238_1,0.0
54779,YAJYINBQFXCAPI-XJGOYTCSSA-N,688239,0.0
54780,YAJYINBQFXCAPI-XJGOYTCSSA-N,688267,0.0


In [12]:
numerical_features = ['Cells_Number_Object_Number']

others=['Cells_Neighbors_FirstClosestObjectNumber_5',
 'Cells_Neighbors_FirstClosestObjectNumber_Adjacent',
 'Cells_Neighbors_SecondClosestObjectNumber_5',
 'Cells_Neighbors_SecondClosestObjectNumber_Adjacent',
 'Cells_Parent_Nuclei',
 'Cytoplasm_Number_Object_Number',
 'Cytoplasm_Parent_Cells',
 'Cytoplasm_Parent_Nuclei',
 'Nuclei_Neighbors_FirstClosestObjectNumber_1',
 'Nuclei_Neighbors_SecondClosestObjectNumber_1',
 'Nuclei_Number_Object_Number']

features = df[["InChIKey"]+numerical_features]
features

,InChIKey,Cells_Number_Object_Number
0,IENZQIKPVFGBNW-UHFFFAOYSA-N,-15.648438
1,GSDSWSVVBLHKDQ-UHFFFAOYSA-N,3.607692
2,CGIGDMFJXJATDK-UHFFFAOYSA-N,-6.164062
3,DSXXEELGXBCYNQ-UHFFFAOYSA-N,-31.828125
4,MYSWGUAQZAJSOK-UHFFFAOYSA-N,3.617188
...,...,...
10521,YAJYINBQFXCAPI-WENCSYSZSA-N,-3.632812
10522,DIBJIDYNEHVBAV-KFRKLSNLSA-N,-3.046875
10523,YAJYINBQFXCAPI-VPYPWEPUSA-N,-1.875000
10524,YAJYINBQFXCAPI-RZIGYZOXSA-N,1.796875


In [13]:
data = pd.merge(melt, features)
data

,InChIKey,Task,Label,Cells_Number_Object_Number
0,IENZQIKPVFGBNW-UHFFFAOYSA-N,737822,0.0,-15.648438
1,IENZQIKPVFGBNW-UHFFFAOYSA-N,737826,0.0,-15.648438
2,IENZQIKPVFGBNW-UHFFFAOYSA-N,737823_1,0.0,-15.648438
3,IENZQIKPVFGBNW-UHFFFAOYSA-N,737824_1,0.0,-15.648438
4,IENZQIKPVFGBNW-UHFFFAOYSA-N,1495242,0.0,-15.648438
...,...,...,...,...
54777,YAJYINBQFXCAPI-RZIGYZOXSA-N,688653,0.0,1.796875
54778,YAJYINBQFXCAPI-RZIGYZOXSA-N,688238_1,0.0,1.796875
54779,YAJYINBQFXCAPI-XJGOYTCSSA-N,688239,0.0,-9.460938
54780,YAJYINBQFXCAPI-XJGOYTCSSA-N,688267,0.0,-9.460938


In [14]:
data.Task.nunique()

213

In [15]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the JSON file containing the task splits
with open('data/output/data_split.json', 'r') as f:
    task_splits = json.load(f)

train_tasks = task_splits['train']
test_tasks = task_splits['test']

# Create DataFrames for each split
train_data = data[data['Task'].isin(train_tasks)]
test_data = data[data['Task'].isin(test_tasks)]

X_train = train_data.drop(columns=['InChIKey', 'Label', 'Task'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['InChIKey', 'Label', 'Task'])
y_test = test_data['Label']

# Save the columns for final DataFrame
inchikey_test = test_data['InChIKey']
task_test = test_data['Task']

# Apply preprocessing
preprocessor = StandardScaler()
X_train = preprocessor.fit_transform(X_train).astype('float32')
X_test = preprocessor.transform(X_test).astype('float32')

# Convert to PyTorch tensors and move to device
X_train_tensor = torch.tensor(X_train).to(device)
y_train_tensor = torch.tensor(y_train.values).to(device)
X_test_tensor = torch.tensor(X_test).to(device)
y_test_tensor = torch.tensor(y_test.values).to(device)

# Create Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Model
class ImprovedNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImprovedNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.5)
        
        self.layer2 = nn.Linear(128, 64)
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.5)
        
        self.layer3 = nn.Linear(64, 32)
        self.batch_norm3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.5)
        
        self.layer4 = nn.Linear(32, output_dim)

    def forward(self, x, weights=None):
        if weights is None:
            x = F.leaky_relu(self.batch_norm1(self.layer1(x)))
            x = self.dropout1(x)
            x = F.leaky_relu(self.batch_norm2(self.layer2(x)))
            x = self.dropout2(x)
            x = F.leaky_relu(self.batch_norm3(self.layer3(x)))
            x = self.dropout3(x)
            x = self.layer4(x)
        else:
            x = F.leaky_relu(F.linear(x, weights['layer1.weight'], weights['layer1.bias']))
            if x.shape[0] > 1:  # Apply batch norm only if batch size > 1
                x = self.batch_norm1(x)
            x = F.dropout(x, p=0.5, training=self.training)
            x = F.leaky_relu(F.linear(x, weights['layer2.weight'], weights['layer2.bias']))
            if x.shape[0] > 1:  # Apply batch norm only if batch size > 1
                x = self.batch_norm2(x)
            x = F.dropout(x, p=0.5, training=self.training)
            x = F.leaky_relu(F.linear(x, weights['layer3.weight'], weights['layer3.bias']))
            if x.shape[0] > 1:  # Apply batch norm only if batch size > 1
                x = self.batch_norm3(x)
            x = F.dropout(x, p=0.5, training=self.training)
            x = F.linear(x, weights['layer4.weight'], weights['layer4.bias'])
        return x

input_dim = X_train.shape[1]
output_dim = 1  # Assuming binary classification
model = ImprovedNN(input_dim, output_dim).to(device)

# Define MAML
class MAML:
    def __init__(self, model, inner_lr=0.01, outer_lr=0.001, inner_steps=1):
        self.model = model
        self.inner_lr = inner_lr
        self.outer_lr = outer_lr
        self.inner_steps = inner_steps
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.outer_lr)

    def inner_loop(self, X, y):
        # Create a copy of the model for inner loop updates
        fast_weights = {name: param.clone() for name, param in self.model.named_parameters()}
        for _ in range(self.inner_steps):
            # Forward pass
            outputs = self.model(X, fast_weights)
            loss = nn.BCEWithLogitsLoss()(outputs, y.float().view(-1, 1))
            # Compute gradients
            gradients = torch.autograd.grad(loss, fast_weights.values(), create_graph=True, allow_unused=True)
            # Update fast weights
            fast_weights = {name: param - self.inner_lr * grad if grad is not None else param
                            for (name, param), grad in zip(fast_weights.items(), gradients)}
        return fast_weights

    def outer_loop(self, task_data):
        meta_loss = 0.0
        all_val_targets = []
        all_val_outputs = []
        for task in task_data:
            X_train, y_train, X_val, y_val = task
            # Perform inner loop
            fast_weights = self.inner_loop(X_train, y_train)
            # Validation with updated weights
            outputs = self.model(X_val, fast_weights)
            loss = nn.BCEWithLogitsLoss()(outputs, y_val.float().view(-1, 1))
            meta_loss += loss
            all_val_targets.append(y_val.cpu().numpy())
            all_val_outputs.append(torch.sigmoid(outputs).detach().cpu().numpy())
        
        # Backward pass and outer update
        self.optimizer.zero_grad()
        meta_loss.backward()
        self.optimizer.step()

        # Calculate AUC-ROC for validation
        all_val_targets = np.concatenate(all_val_targets)
        all_val_outputs = np.concatenate(all_val_outputs)
        auc_roc = roc_auc_score(all_val_targets, all_val_outputs)
        return meta_loss.item(), auc_roc

# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)

# Training
maml = MAML(model)

for epoch in range(25):  # Adjust the number of epochs as needed
    task_data = []  # Prepare your task data here
    for task in train_tasks:
        task_subset = train_data[train_data['Task'] == task]
        X_task = torch.tensor(preprocessor.transform(task_subset.drop(columns=['InChIKey', 'Label', 'Task'])).astype('float32')).to(device)
        y_task = torch.tensor(task_subset['Label'].values).to(device)
        # Split into inner and outer task data, stratified by the labels
        X_task_train, X_task_val, y_task_train, y_task_val = train_test_split(X_task.cpu(), y_task.cpu(), test_size=0.2, stratify=y_task.cpu())
        X_task_train, y_task_train = X_task_train.to(device), y_task_train.to(device)
        X_task_val, y_task_val = X_task_val.to(device), y_task_val.to(device)
        task_data.append((X_task_train, y_task_train, X_task_val, y_task_val))
    
    meta_loss, auc_roc = maml.outer_loop(task_data)
    print(f"Epoch {epoch + 1}, Meta Loss: {meta_loss}, AUC-ROC: {auc_roc}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities = torch.sigmoid(outputs)
    predictions = probabilities.round()
    accuracy = (predictions.view(-1) == y_test_tensor).float().mean().item()
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Convert to NumPy arrays
probabilities_np = probabilities.cpu().numpy().flatten()
predictions_np = predictions.cpu().numpy().flatten()
y_test_np = y_test_tensor.cpu().numpy()

# Create a DataFrame for the final output
results_df = pd.DataFrame({
    'InChIKey': inchikey_test,
    'Task': task_test,
    'Actual': y_test_np,
    'Predicted': predictions_np,
    'Probability': probabilities_np
})

# Calculate additional metrics
balanced_acc = balanced_accuracy_score(results_df['Actual'], results_df['Predicted'])
precision = precision_score(results_df['Actual'], results_df['Predicted'])
recall = recall_score(results_df['Actual'], results_df['Predicted'])

# Display the DataFrame and metrics
print(results_df)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")


Epoch 1, Meta Loss: 125.56090545654297, AUC-ROC: 0.5104977135784535
Epoch 2, Meta Loss: 119.8109130859375, AUC-ROC: 0.5361293607703354
Epoch 3, Meta Loss: 120.05656433105469, AUC-ROC: 0.5447288638171264
Epoch 4, Meta Loss: 118.9134750366211, AUC-ROC: 0.5393572448039166
Epoch 5, Meta Loss: 118.05323791503906, AUC-ROC: 0.5592652078595011
Epoch 6, Meta Loss: 117.18492126464844, AUC-ROC: 0.5568764961794156
Epoch 7, Meta Loss: 115.08232116699219, AUC-ROC: 0.5711220377719413
Epoch 8, Meta Loss: 115.52130889892578, AUC-ROC: 0.5740504961603718
Epoch 9, Meta Loss: 113.72640991210938, AUC-ROC: 0.5780854462042437
Epoch 10, Meta Loss: 113.55638122558594, AUC-ROC: 0.5711414690906935
Epoch 11, Meta Loss: 111.18592834472656, AUC-ROC: 0.574544146836641
Epoch 12, Meta Loss: 113.1148452758789, AUC-ROC: 0.5710379316222847
Epoch 13, Meta Loss: 110.18997192382812, AUC-ROC: 0.5789061121940012
Epoch 14, Meta Loss: 113.5162353515625, AUC-ROC: 0.5719492928116061
Epoch 15, Meta Loss: 113.06285095214844, AUC-ROC

In [16]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, roc_auc_score

# Assuming `results_df` is already created as shown in your code

# Initialize a list to store the results
metrics_list = []

for task in results_df.Task.unique():
    subdata = results_df[results_df["Task"] == task]
    auc = roc_auc_score(subdata['Actual'], subdata['Probability'])
    balanced_acc = balanced_accuracy_score(subdata['Actual'], subdata['Predicted'])
    precision = precision_score(subdata['Actual'], subdata['Predicted'])
    recall = recall_score(subdata['Actual'], subdata['Predicted'])

    # Print the results
    print(f"Task: {task}")
    print(f"AUC: {auc:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print("-" * 60)

    # Append the results to the list
    metrics_list.append({
        'Task': task,
        'AUC': auc,
        'Balanced Accuracy': balanced_acc,
        'Precision': precision,
        'Recall': recall
    })

# Convert the list of dictionaries to a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Display the DataFrame
print(metrics_df)

# Save the DataFrame to a CSV file
metrics_df.to_csv('task_metrics.csv', index=False)


Task: 737826
AUC: 0.44
Balanced Accuracy: 0.46
Precision: 0.31
Recall: 0.81
------------------------------------------------------------
Task: 737824_1
AUC: 0.48
Balanced Accuracy: 0.49
Precision: 0.40
Recall: 0.86
------------------------------------------------------------
Task: 1495405
AUC: 0.61
Balanced Accuracy: 0.52
Precision: 0.34
Recall: 0.91
------------------------------------------------------------
Task: 737825
AUC: 0.53
Balanced Accuracy: 0.51
Precision: 0.36
Recall: 0.87
------------------------------------------------------------
Task: 752594
AUC: 0.73
Balanced Accuracy: 0.55
Precision: 0.47
Recall: 0.95
------------------------------------------------------------
Task: 600886
AUC: 0.55
Balanced Accuracy: 0.52
Precision: 0.41
Recall: 0.85
------------------------------------------------------------
Task: 752347
AUC: 0.57
Balanced Accuracy: 0.51
Precision: 0.70
Recall: 0.89
------------------------------------------------------------
Task: 809095
AUC: 0.61
Balanced Accura

In [11]:
train_data

,InChIKey,Task,Label,Cells_Number_Object_Number,Cells_Neighbors_FirstClosestObjectNumber_5,Cells_Neighbors_FirstClosestObjectNumber_Adjacent,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number
0,IENZQIKPVFGBNW-UHFFFAOYSA-N,737822,0.0,-15.648438,-15.605469,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
2,IENZQIKPVFGBNW-UHFFFAOYSA-N,737823_1,0.0,-15.648438,-15.605469,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
4,IENZQIKPVFGBNW-UHFFFAOYSA-N,1495242,0.0,-15.648438,-15.605469,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
5,IENZQIKPVFGBNW-UHFFFAOYSA-N,1495404,0.0,-15.648438,-15.605469,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
7,IENZQIKPVFGBNW-UHFFFAOYSA-N,1495415,1.0,-15.648438,-15.605469,-15.605469,-14.804688,-14.804688,-15.648438,-15.648438,-15.648438,-15.648438,-16.062500,-15.687500,-15.648438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54776,YAJYINBQFXCAPI-RZIGYZOXSA-N,688238,0.0,1.796875,0.820312,0.820312,0.843750,0.843750,1.796875,1.796875,1.796875,1.796875,1.101562,1.281250,1.796875
54777,YAJYINBQFXCAPI-RZIGYZOXSA-N,688653,0.0,1.796875,0.820312,0.820312,0.843750,0.843750,1.796875,1.796875,1.796875,1.796875,1.101562,1.281250,1.796875
54778,YAJYINBQFXCAPI-RZIGYZOXSA-N,688238_1,0.0,1.796875,0.820312,0.820312,0.843750,0.843750,1.796875,1.796875,1.796875,1.796875,1.101562,1.281250,1.796875
54779,YAJYINBQFXCAPI-XJGOYTCSSA-N,688239,0.0,-9.460938,-9.492188,-9.492188,-9.835938,-9.835938,-9.453125,-9.460938,-9.453125,-9.453125,-9.898438,-9.359375,-9.460938


In [ ]:
# This is a backup simpler NN that also works!


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score

# Load the JSON file containing the task splits
with open('data/output/data_split.json', 'r') as f:
    task_splits = json.load(f)

train_tasks = task_splits['train']
test_tasks = task_splits['test']

# Create DataFrames for each split
train_data = data[data['Task'].isin(train_tasks)]
test_data = data[data['Task'].isin(test_tasks)]

X_train = train_data.drop(columns=['InChIKey', 'Label', 'Task'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['InChIKey', 'Label', 'Task'])
y_test = test_data['Label']

# Save the columns for final DataFrame
inchikey_test = test_data['InChIKey']
task_test = test_data['Task']

# Apply preprocessing
preprocessor = StandardScaler()
X_train = preprocessor.fit_transform(X_train).astype('float32')
X_test = preprocessor.transform(X_test).astype('float32')

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train.values)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test.values)

# Create Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, output_dim)

    def forward(self, x, weights=None):
        if weights is None:
            x = torch.relu(self.layer1(x))
            x = torch.relu(self.layer2(x))
            x = self.layer3(x)
        else:
            x = torch.relu(F.linear(x, weights['layer1.weight'], weights['layer1.bias']))
            x = torch.relu(F.linear(x, weights['layer2.weight'], weights['layer2.bias']))
            x = F.linear(x, weights['layer3.weight'], weights['layer3.bias'])
        return x

input_dim = X_train.shape[1]
output_dim = 1  # Assuming binary classification
model = SimpleNN(input_dim, output_dim)

# Define MAML
class MAML:
    def __init__(self, model, inner_lr=0.01, outer_lr=0.001, inner_steps=1):
        self.model = model
        self.inner_lr = inner_lr
        self.outer_lr = outer_lr
        self.inner_steps = inner_steps
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.outer_lr)

    def inner_loop(self, X, y):
        # Create a copy of the model for inner loop updates
        fast_weights = {name: param.clone() for name, param in self.model.named_parameters()}
        for _ in range(self.inner_steps):
            # Forward pass
            outputs = self.model(X, fast_weights)
            loss = nn.BCEWithLogitsLoss()(outputs, y.float().view(-1, 1))
            # Compute gradients
            gradients = torch.autograd.grad(loss, fast_weights.values(), create_graph=True)
            # Update fast weights
            fast_weights = {name: param - self.inner_lr * grad for (name, param), grad in zip(fast_weights.items(), gradients)}
        return fast_weights

    def outer_loop(self, task_data):
        meta_loss = 0.0
        for task in task_data:
            X_train, y_train, X_val, y_val = task
            # Perform inner loop
            fast_weights = self.inner_loop(X_train, y_train)
            # Validation with updated weights
            outputs = self.model(X_val, fast_weights)
            loss = nn.BCEWithLogitsLoss()(outputs, y_val.float().view(-1, 1))
            meta_loss += loss
        # Backward pass and outer update
        self.optimizer.zero_grad()
        meta_loss.backward()
        self.optimizer.step()
        return meta_loss.item()

# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)

# Training
maml = MAML(model)

for epoch in range(100):  # Adjust the number of epochs as needed
    task_data = []  # Prepare your task data here
    for task in train_tasks:
        task_subset = train_data[train_data['Task'] == task]
        X_task = torch.tensor(preprocessor.transform(task_subset.drop(columns=['InChIKey', 'Label', 'Task'])).astype('float32'))
        y_task = torch.tensor(task_subset['Label'].values)
        # Split into inner and outer task data
        X_task_train, X_task_val, y_task_train, y_task_val = train_test_split(X_task, y_task, test_size=0.2)
        task_data.append((X_task_train, y_task_train, X_task_val, y_task_val))
    
    meta_loss = maml.outer_loop(task_data)
    print(f"Epoch {epoch + 1}, Meta Loss: {meta_loss}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities = torch.sigmoid(outputs)
    predictions = probabilities.round()
    accuracy = (predictions.view(-1) == y_test_tensor).float().mean().item()
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Convert to NumPy arrays
probabilities_np = probabilities.numpy().flatten()
predictions_np = predictions.numpy().flatten()
y_test_np = y_test_tensor.numpy()

# Create a DataFrame for the final output
results_df = pd.DataFrame({
    'InChIKey': inchikey_test,
    'Task': task_test,
    'Actual': y_test_np,
    'Predicted': predictions_np,
    'Probability': probabilities_np
})

# Calculate additional metrics
balanced_acc = balanced_accuracy_score(results_df['Actual'], results_df['Predicted'])
precision = precision_score(results_df['Actual'], results_df['Predicted'])
recall = recall_score(results_df['Actual'], results_df['Predicted'])

# Display the DataFrame and metrics
print(results_df)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")

# Per-task metrics
for task in results_df.Task.unique():
    subdata = results_df[results_df["Task"] == task]
    balanced_acc = balanced_accuracy_score(subdata['Actual'], subdata['Predicted'])
    precision = precision_score(subdata['Actual'], subdata['Predicted'])
    recall = recall_score(subdata['Actual'], subdata['Predicted'])
    class_report = classification_report(subdata['Actual'], subdata['Predicted'])
    conf_matrix = confusion_matrix(subdata['Actual'], subdata['Predicted'])
    
    print(f"Task: {task}")
    print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print("Classification Report:")
    print(class_report)
    print("Confusion Matrix:")
    print(conf_matrix)
    print("-" * 60)


In [ ]:
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

for task in results_df.Task.unique():
    subdata = results_df[results_df["Task"] == task]
    auc = roc_auc_score(subdata['Actual'], subdata['Probability'])
    
    print(f"Task: {task}")
    print(f"AUC: {auc:.2f}")

    print("-" * 60)


In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, roc_auc_score

# Assuming `results_df` is already created as shown in your code

# Initialize a list to store the results
metrics_list = []

for task in results_df.Task.unique():
    subdata = results_df[results_df["Task"] == task]
    auc = roc_auc_score(subdata['Actual'], subdata['Probability'])
    balanced_acc = balanced_accuracy_score(subdata['Actual'], subdata['Predicted'])
    precision = precision_score(subdata['Actual'], subdata['Predicted'])
    recall = recall_score(subdata['Actual'], subdata['Predicted'])

    # Print the results
    print(f"Task: {task}")
    print(f"AUC: {auc:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print("-" * 60)

    # Append the results to the list
    metrics_list.append({
        'Task': task,
        'AUC': auc,
        'Balanced Accuracy': balanced_acc,
        'Precision': precision,
        'Recall': recall
    })

# Convert the list of dictionaries to a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Display the DataFrame
print(metrics_df)

# Save the DataFrame to a CSV file
# metrics_df.to_csv('task_metrics.csv', index=False)
